In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
#loading orders data from disk
orders=pd.read_csv('orders.csv')
print(orders.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')


In [3]:
#loading order and products data for prior set
order_products_prior=pd.read_csv('order_products__prior.csv')
print(order_products_prior.columns)

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')


In [4]:
#obtaining user, order and product info for prior set
user_order_products=pd.merge(orders[orders.eval_set=='prior'],order_products_prior,on='order_id',how='left')
print(user_order_products.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered'],
      dtype='object')


In [5]:
#loading detailed product information
products=pd.read_csv('products.csv')

In [6]:
#obtaining user, order and product detailed info for prior set
user_order_products_all_details=pd.merge(user_order_products,products,on='product_id',how='left')
user_order_products_all_details.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id'],
      dtype='object')

# Extracting Features from Prior set

# User, Product Features

In [34]:
#extracting user_product_cnt, the number of times a given user has ordered a given product in the prior set
user_product_cnt=user_order_products_all_details.groupby(['user_id','product_id']).size().reset_index()
user_product_cnt['user_product_cnt']=user_product_cnt[0]
user_product_cnt.drop([0],axis=1,inplace=True)
user_product_cnt.head(5)


,user_id,product_id,user_product_cnt
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [35]:
#saving user_product_cnt feature to disk
user_product_cnt.to_csv('user_product_cnt.csv',index=False)

In [36]:
#extracting the count of all products for a user
user_all_product_cnt=user_product_cnt.groupby(['user_id']).agg({'user_product_cnt':sum})
user_all_product_cnt['user_all_product_cnt']=user_all_product_cnt['user_product_cnt']
user_all_product_cnt.drop(['user_product_cnt'],axis=1,inplace=True)
user_all_product_cnt.head(5)

,user_all_product_cnt
user_id,
1,59
2,195
3,88
4,18
5,37


In [37]:
#extracting the proportion of a product being a ordered by a user out all of the products the user has ordered
user_product_prop=pd.merge(user_product_cnt,user_all_product_cnt,on='user_id',how='left')
user_product_prop['user_product_prop']=user_product_prop['user_product_cnt']/user_product_prop['user_all_product_cnt']
user_product_prop.drop(['user_product_cnt','user_all_product_cnt'],axis=1,inplace=True)
user_product_prop

,user_id,product_id,user_product_prop
0,1,196,0.169492
1,1,10258,0.152542
2,1,10326,0.016949
3,1,12427,0.169492
4,1,13032,0.050847
...,...,...,...
13307948,206209,43961,0.023256
13307949,206209,44325,0.007752
13307950,206209,48370,0.007752
13307951,206209,48697,0.007752


In [38]:
#saving user_all_product_cnt feature to disk
user_product_prop.to_csv('user_product_prop.csv',index=False)

In [39]:
#extracting user_product_dow_mean, the mean of day of the week for a user and a product
user_product_dow_mean=user_order_products_all_details.groupby(['user_id','product_id']).agg({'order_dow':'mean'}).reset_index()
user_product_dow_mean.columns=['user_id','product_id','user_product_dow_mean']
user_product_dow_mean

,user_id,product_id,user_product_dow_mean
0,1,196,2.500000
1,1,10258,2.555556
2,1,10326,4.000000
3,1,12427,2.500000
4,1,13032,2.666667
...,...,...,...
13307948,206209,43961,1.666667
13307949,206209,44325,2.000000
13307950,206209,48370,4.000000
13307951,206209,48697,2.000000


In [40]:
#saving user_product_dow_mean feature to disk
user_product_dow_mean.to_csv('user_product_dow_mean.csv',index=False)

In [41]:
#extracting user_product_dow_mean, the mean of day of the week for a user and a product
user_product_hod_mean=user_order_products_all_details.groupby(['user_id','product_id']).agg({'order_hour_of_day':'mean'}).reset_index()
user_product_hod_mean.columns=['user_id','product_id','user_product_hod_mean']
user_product_hod_mean

,user_id,product_id,user_product_hod_mean
0,1,196,10.300000
1,1,10258,10.555556
2,1,10326,15.000000
3,1,12427,10.300000
4,1,13032,8.000000
...,...,...,...
13307948,206209,43961,13.000000
13307949,206209,44325,13.000000
13307950,206209,48370,10.000000
13307951,206209,48697,13.000000


In [42]:
#saving user_product_hod_mean feature to disk
user_product_hod_mean.to_csv('user_product_hod_mean.csv',index=False)

In [55]:
#extracting user_product_atc_mode, the mode of add_to_cart_order for a product and a user
user_product_atc_mode=user_order_products_all_details.groupby(['user_id','product_id','add_to_cart_order']).agg({'reordered':'count'}).reset_index()
atc_max=user_product_atc_mode.groupby(['user_id','product_id']).agg({'reordered':max}).reset_index()
atc_max.columns=['user_id','product_id','atc_max']
user_product_atc_mode=pd.merge(user_product_atc_mode,atc_max,on=['user_id','product_id'],how='left')
user_product_atc_mode=user_product_atc_mode[user_product_atc_mode.reordered==user_product_atc_mode.atc_max].reset_index()[['user_id','product_id','add_to_cart_order']]
user_product_atc_mode.columns=['user_id','product_id','user_product_atc_mode']
user_product_atc_mode

,user_id,product_id,user_product_atc_mode
0,1,196,1
1,1,10258,3
2,1,10326,5
3,1,12427,2
4,1,13032,5
...,...,...,...
19576352,206209,44325,8
19576353,206209,48370,8
19576354,206209,48697,6
19576355,206209,48742,5


In [57]:
#Some of the users order some products at multiple orders in the cart equally, hence taking minimum of those
user_product_atc_mode_min=user_product_atc_mode.groupby(['user_id','product_id']).agg({'user_product_atc_mode':min}).reset_index()
user_product_atc_mode_min.columns=['user_id','product_id','user_product_atc_mode_min']
user_product_atc_mode_min.head(5)

,user_id,product_id,user_product_atc_mode_min
0,1,196,1
1,1,10258,3
2,1,10326,5
3,1,12427,2
4,1,13032,5


In [58]:
#Some of the users order some products at multiple orders in the cart equally, hence taking maximum of those
user_product_atc_mode_max=user_product_atc_mode.groupby(['user_id','product_id']).agg({'user_product_atc_mode':max}).reset_index()
user_product_atc_mode_max.columns=['user_id','product_id','user_product_atc_mode_max']
user_product_atc_mode_max.head(5)

,user_id,product_id,user_product_atc_mode_max
0,1,196,1
1,1,10258,3
2,1,10326,5
3,1,12427,2
4,1,13032,8


In [44]:
#extracting atc_mean, the mean of add_to_cart_order for a user_id and product_id
user_product_atc_mean=user_order_products_all_details.groupby(['user_id','product_id']).agg({'add_to_cart_order':'mean'}).reset_index()
user_product_atc_mean.columns=['user_id','product_id','user_product_atc_mean']
user_product_atc_min=user_order_products_all_details.groupby(['user_id','product_id']).agg({'add_to_cart_order':min}).reset_index()
user_product_atc_min.columns=['user_id','product_id','user_product_atc_min']
user_product_atc_max=user_order_products_all_details.groupby(['user_id','product_id']).agg({'add_to_cart_order':max}).reset_index()
user_product_atc_max.columns=['user_id','product_id','user_product_atc_max']
user_product_atc_max

,user_id,product_id,user_product_atc_max
0,1,196,4
1,1,10258,6
2,1,10326,5
3,1,12427,9
4,1,13032,8
...,...,...,...
13307948,206209,43961,12
13307949,206209,44325,8
13307950,206209,48370,8
13307951,206209,48697,6


In [45]:
#idea inspired from https://medium.com/kaggle-blog/instacart-market-basket-analysis-feda2700cded
#extracting the number of days passed after the user last ordered the product 
#adding cumulative days_since_prior_order for each user
orders_prior=orders[orders.eval_set=='prior']
orders_prior['days_since_prior_order_cum_sum']=orders_prior.groupby('user_id').agg({'days_since_prior_order':'cumsum'})
orders_prior.fillna(0,inplace=True)

#extracting max value of days_since_prior_order_cum_sum for each user
max_cumsum=orders_prior.groupby('user_id').agg({'days_since_prior_order_cum_sum':max}).reset_index()
max_cumsum.columns=['user_id','max_cum_sum']

#finding the number of days that has passed after each order by a user
orders_prior1=pd.merge(orders_prior,max_cumsum,on='user_id',how='left')
orders_prior1['days_since_product']=orders_prior1['max_cum_sum']-orders_prior1['days_since_prior_order_cum_sum']

#extracting the number of days that has passed after the user has recently ordered the product
user_days_since_product=pd.merge(orders_prior1,order_products_prior[['order_id','product_id']],on='order_id')[['user_id','product_id','days_since_product']]
user_days_since_product=user_days_since_product.groupby(['user_id','product_id']).agg({'days_since_product':min}).reset_index()
user_days_since_product.head()

,user_id,product_id,days_since_product
0,1,196,0.0
1,1,10258,0.0
2,1,10326,83.0
3,1,12427,0.0
4,1,13032,0.0


In [46]:
#saving user_days_since_product data to disk
user_days_since_product.to_csv('user_days_since_product.csv',index=False)

In [2]:
#combining all user_product_features
user_product_features=pd.DataFrame({'user_id':user_days_since_product.user_id,
                                   'product_id':user_days_since_product.product_id,
                                   'user_days_since_product':user_days_since_product.days_since_product,
                                    'user_product_hod_mean':user_product_hod_mean.user_product_hod_mean,
                                    'user_product_dow_mean':user_product_dow_mean.user_product_dow_mean,
                                    'user_product_prop':user_product_prop.user_product_prop,
                                    'user_product_cnt':user_product_cnt.user_product_cnt,
                                    'user_product_atc_mode_min':user_product_atc_mode_min.user_product_atc_mode_min,
                                    'user_product_atc_mode_max':user_product_atc_mode_max.user_product_atc_mode_max,
                                    'user_product_atc_min':user_product_atc_min.user_product_atc_min,
                                    'user_product_atc_max':user_product_atc_max.user_product_atc_max,
                                    'user_product_atc_mean':user_product_atc_mean.user_product_atc_mean
                                   })


In [63]:
#saving user_product_features to disk
user_product_features.to_csv('user_product_features.csv',index=False)

# Extracting user features

In [11]:
#Extracting user_max_ono, the maximum order number for a user
user_max_ono=user_order_products_all_details.groupby(['user_id']).agg({'order_number':max}).reset_index()
user_max_ono.columns=['user_id','user_max_ono']
user_max_ono

,user_id,user_max_ono
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4
...,...,...
206204,206205,3
206205,206206,67
206206,206207,16
206207,206208,49


In [12]:
#Extracting user_sum_reord, the number of reorders by a user
user_sum_reord=user_order_products_all_details.groupby(['user_id']).agg({'reordered':sum}).reset_index()
user_sum_reord.columns=['user_id','user_sum_reord']
user_sum_reord

,user_id,user_sum_reord
0,1,41
1,2,93
2,3,55
3,4,1
4,5,14
...,...,...
206204,206205,8
206205,206206,135
206206,206207,131
206207,206208,479


In [13]:
#Extracting user_reord_prop, the number of reorders by a user out of total purchases
user_reord_prop=pd.DataFrame(user_sum_reord)
user_reord_prop['user_count_reord']=user_order_products_all_details.groupby(['user_id']).agg({'reordered':'count'}).reset_index().reordered
user_reord_prop['user_reord_prop']=user_reord_prop['user_sum_reord']/user_reord_prop['user_count_reord']
user_reord_prop.drop(['user_sum_reord','user_count_reord'],axis=1,inplace=True)
user_reord_prop

,user_id,user_reord_prop
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378
...,...,...
206204,206205,0.250000
206205,206206,0.473684
206206,206207,0.587444
206207,206208,0.707533


In [14]:
#extracting user_prod_reord_prop, the proportion of unique products that are reordered atleast once out of all unique products of a user
user_prod_reord_prop=user_order_products_all_details.groupby(['user_id','product_id']).agg({'reordered':sum}).reset_index()
user_prod_reord_prop['reordered']=user_prod_reord_prop['reordered'].apply(lambda x: 1 if x>0 else 0)
user_prod_reord_prop=user_prod_reord_prop.groupby(['user_id']).agg({'product_id':'count','reordered':sum}).reset_index()
user_prod_reord_prop['user_prod_reord_prop']=user_prod_reord_prop['reordered']/user_prod_reord_prop['product_id']
user_prod_reord_prop.drop(['product_id','reordered'],axis=1,inplace=True)
user_prod_reord_prop

,user_id,user_prod_reord_prop
0,1,0.555556
1,2,0.362745
2,3,0.575758
3,4,0.058824
4,5,0.347826
...,...,...
206204,206205,0.291667
206205,206206,0.293333
206206,206207,0.521739
206207,206208,0.479798


In [15]:
#extracting user_uniqpr, the number of unique products each user has ordered
user_uniqpr=user_order_products_all_details.groupby(['user_id']).agg({'product_id':'nunique'}).reset_index()
user_uniqpr.columns=['user_id','user_uniq_pr']
user_uniqpr.head(5)

,user_id,user_uniq_pr
0,1,18
1,2,102
2,3,33
3,4,17
4,5,23


In [16]:
#extracting user_uniqpr_prop, the proportion of unique products for each user
user_total_products=user_order_products_all_details.groupby(['user_id']).agg({'product_id':'count'}).reset_index().product_id
user_uniqpr_prop=pd.DataFrame(user_uniqpr)
user_uniqpr_prop['total_products']=user_total_products
user_uniqpr_prop['user_uniqpr_prop']=user_uniqpr_prop['user_uniq_pr']/user_uniqpr_prop['total_products']
user_uniqpr_prop.drop(['user_uniq_pr','total_products'],axis=1,inplace=True)
user_uniqpr_prop.head(5)

,user_id,user_uniqpr_prop
0,1,0.305085
1,2,0.523077
2,3,0.375000
3,4,0.944444
4,5,0.621622


In [17]:
#extracting the proportion of orders of a user that contain atleast one reorder
user_order_reord_prop=user_order_products_all_details.groupby(['user_id','order_number']).agg({'reordered':sum}).reset_index()
user_order_reord_prop['reordered']=user_order_reord_prop['reordered'].apply(lambda x: 1 if x>0 else 0)
user_order_reord_prop=user_order_reord_prop.groupby(['user_id']).agg({'order_number':'count','reordered':sum}).reset_index()
user_order_reord_prop['user_order_reord_prop']=user_order_reord_prop['reordered']/user_order_reord_prop['order_number']
user_order_reord_prop.drop(['order_number','reordered'],axis=1,inplace=True)
user_order_reord_prop

,user_id,user_order_reord_prop
0,1,0.900000
1,2,0.857143
2,3,0.916667
3,4,0.200000
4,5,0.750000
...,...,...
206204,206205,0.666667
206205,206206,0.940299
206206,206207,0.937500
206207,206208,0.979592


In [18]:
#extracting user_dsp_mean, the mean of days_since_prior_order for each user id
user_dsp_mean=user_order_products_all_details.groupby(['user_id']).agg({'days_since_prior_order':'mean'}).reset_index()
user_dsp_mean.columns=['user_id','user_dsp_mean']
user_dsp_mean

,user_id,user_dsp_mean
0,1,20.259259
1,2,15.967033
2,3,11.487179
3,4,15.357143
4,5,14.500000
...,...,...
206204,206205,20.666667
206205,206206,4.042705
206206,206207,14.879397
206207,206208,7.442105


In [19]:
#idea inspired from https://medium.com/kaggle-blog/instacart-market-basket-analysis-feda2700cded
#extracting user_min_order_size,user_max_orders_size,user_mean_order_size, the order size statistics for each user
user_order_sizes=user_order_products.groupby(['user_id','order_number']).agg({'product_id':'count'}).reset_index()
user_min_order_size=user_order_sizes.groupby(['user_id']).agg({'product_id':min}).reset_index()
user_min_order_size.columns=['user_id','user_min_order_size']
user_max_order_size=user_order_sizes.groupby(['user_id']).agg({'product_id':max}).reset_index()
user_max_order_size.columns=['user_id','user_max_order_size']
user_mean_order_size=user_order_sizes.groupby(['user_id']).agg({'product_id':'mean'}).reset_index()
user_mean_order_size.columns=['user_id','user_mean_order_size']
user_mean_order_size

,user_id,user_mean_order_size
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000
...,...,...
206204,206205,10.666667
206205,206206,4.253731
206206,206207,13.937500
206207,206208,13.816327


In [20]:
#combining all the user related features
user_features=pd.DataFrame({'user_id':user_max_ono.user_id,'user_max_ono':user_max_ono.user_max_ono,
                           'user_sum_reord':user_sum_reord.user_sum_reord,
                            'user_reord_prop':user_reord_prop.user_reord_prop,
                            'user_prod_reord_prop':user_prod_reord_prop.user_prod_reord_prop,
                           'user_uniqpr':user_uniqpr.user_uniq_pr,
                           'user_uniqpr_prop':user_uniqpr_prop.user_uniqpr_prop,
                           'user_order_reord_prop':user_order_reord_prop.user_order_reord_prop,
                           'user_dsp_mean':user_dsp_mean.user_dsp_mean,
                           'user_min_order_size':user_min_order_size.user_min_order_size,
                           'user_max_order_size':user_max_order_size.user_max_order_size,
                           'user_mean_order_size':user_mean_order_size.user_mean_order_size})
user_features.head(5)

,user_id,user_max_ono,user_sum_reord,user_reord_prop,user_prod_reord_prop,user_uniqpr,user_uniqpr_prop,user_order_reord_prop,user_dsp_mean,user_min_order_size,user_max_order_size,user_mean_order_size
0,1,10,41,0.694915,0.555556,18,0.305085,0.900000,20.259259,4,9,5.900000
1,2,14,93,0.476923,0.362745,102,0.523077,0.857143,15.967033,5,26,13.928571
2,3,12,55,0.625000,0.575758,33,0.375000,0.916667,11.487179,5,11,7.333333
3,4,5,1,0.055556,0.058824,17,0.944444,0.200000,15.357143,2,7,3.600000
4,5,4,14,0.378378,0.347826,23,0.621622,0.750000,14.500000,5,12,9.250000


In [22]:
#storing user_features to disk
user_features.to_csv('user_features.csv',index=False)

# Extracting Product features

In [30]:
#idea inspired from https://medium.com/kaggle-blog/instacart-market-basket-analysis-feda2700cded
#extracting product_ratios_users_oneshot, i.e the number of users that have ordered the item only once
product_ratios_users_oneshot=user_order_products.groupby(['user_id','product_id']).agg({'reordered':'count'}).reset_index()
product_ratios_users_oneshot['reordered']=product_ratios_users_oneshot['reordered'].apply(lambda x: 1 if x==1 else 0)
product_ratios_users_oneshot=product_ratios_users_oneshot.groupby(['product_id']).agg({'user_id':'count','reordered':sum}).reset_index()
product_ratios_users_oneshot['product_ratios_users_oneshot']=product_ratios_users_oneshot['reordered']/product_ratios_users_oneshot['user_id']
product_ratios_users_oneshot.drop(['user_id','reordered'],axis=1,inplace=True)
product_ratios_users_oneshot

,product_id,product_ratios_users_oneshot
0,1,0.614525
1,2,0.897436
2,3,0.513514
3,4,0.648352
4,5,0.333333
...,...,...
49672,49684,0.875000
49673,49685,0.860465
49674,49686,0.555556
49675,49687,0.428571


In [31]:
#extracting product_cart_mean, the average of add_to_card_order for a product
product_cart_mean=user_order_products.groupby('product_id').agg({'add_to_cart_order':'mean'}).reset_index()
product_cart_mean.columns=['product_id','product_cart_mean']
product_cart_mean.head(5)

,product_id,product_cart_mean
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [32]:
#extracting product_reord_count, the number of times a product has been reordered across all users
product_reord_count=user_order_products.groupby(['product_id']).agg({'reordered':sum}).reset_index()
product_reord_count.columns=['product_id','product_reord_count']
product_reord_count

,product_id,product_reord_count
0,1,1136
1,2,12
2,3,203
3,4,147
4,5,9
...,...,...
49672,49684,1
49673,49685,6
49674,49686,84
49675,49687,6


In [33]:
#extracting product_reord_prop, the proportion of number of reorders for a product out all purchases of it
product_reord_prop=pd.DataFrame(product_reord_count)
product_reord_prop['reord_count']=user_order_products.groupby(['product_id']).agg({'reordered':'count'}).reset_index().reordered
product_reord_prop['product_reord_prop']=product_reord_prop['product_reord_count']/product_reord_prop['reord_count']
product_reord_prop.drop(['product_reord_count','reord_count'],axis=1,inplace=True)
product_reord_prop

,product_id,product_reord_prop
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000
...,...,...
49672,49684,0.111111
49673,49685,0.122449
49674,49686,0.700000
49675,49687,0.461538


In [34]:
#extracting prod_uniq_us, the number of unique users for each product
prod_uniq_us=user_order_products.groupby('product_id').agg({'user_id':'nunique'}).reset_index()
prod_uniq_us.columns=['product_id','prod_uniq_us']
prod_uniq_us.head(5)


,product_id,prod_uniq_us
0,1,716
1,2,78
2,3,74
3,4,182
4,5,6


In [35]:
#extracting prod_uniq_us_prop, the number of unique users out of total purchases of a product
prod_uniq_us_prop=pd.DataFrame(prod_uniq_us)
prod_uniq_us_prop['total_purchases']=user_order_products.groupby('product_id').agg({'reordered':'count'}).reset_index().reordered
prod_uniq_us_prop['prod_uniq_us_prop']=prod_uniq_us_prop['prod_uniq_us']/prod_uniq_us_prop['total_purchases']
prod_uniq_us_prop.drop(['prod_uniq_us','total_purchases'],axis=1,inplace=True)
prod_uniq_us_prop.head(5)

,product_id,prod_uniq_us_prop
0,1,0.386609
1,2,0.866667
2,3,0.267148
3,4,0.553191
4,5,0.400000


In [36]:
#prod_us_reord_prop, the ratio of unique users that have reordered the item atleast once
prod_us_reord_prop=user_order_products.groupby(['product_id','user_id']).agg({'reordered':sum}).reset_index()
prod_us_reord_prop['reordered']=prod_us_reord_prop['reordered'].apply(lambda x: 1 if x>0 else 0)
prod_us_reord_prop=prod_us_reord_prop.groupby(['product_id']).agg({'user_id':'count','reordered':sum}).reset_index()
prod_us_reord_prop['prod_us_reord_prop']=prod_us_reord_prop['reordered']/prod_us_reord_prop['user_id']
prod_us_reord_prop.drop(['user_id','reordered'],axis=1,inplace=True)
prod_us_reord_prop

,product_id,prod_us_reord_prop
0,1,0.385475
1,2,0.102564
2,3,0.486486
3,4,0.351648
4,5,0.666667
...,...,...
49672,49684,0.125000
49673,49685,0.139535
49674,49686,0.444444
49675,49687,0.571429


In [37]:
#combining all product_features
product_features=pd.DataFrame({'product_id':product_ratios_users_oneshot.product_id,
                              'product_ratios_users_oneshot':product_ratios_users_oneshot.product_ratios_users_oneshot,
                              'product_cart_mean':product_cart_mean.product_cart_mean,
                               'product_reord_count':product_reord_count.product_reord_count,
                               'product_reord_prop':product_reord_prop.product_reord_prop,
                               'prod_uniq_us':prod_uniq_us.prod_uniq_us,
                               'prod_uniq_us_prop':prod_uniq_us_prop.prod_uniq_us_prop,
                               'prod_us_reord_prop':prod_us_reord_prop.prod_us_reord_prop
                              })
product_features.head(5)

,product_id,product_ratios_users_oneshot,product_cart_mean,product_reord_count,product_reord_prop,prod_uniq_us,prod_uniq_us_prop,prod_us_reord_prop
0,1,0.614525,5.801836,1136,0.613391,716,0.386609,0.385475
1,2,0.897436,9.888889,12,0.133333,78,0.866667,0.102564
2,3,0.513514,6.415162,203,0.732852,74,0.267148,0.486486
3,4,0.648352,9.507599,147,0.446809,182,0.553191,0.351648
4,5,0.333333,6.466667,9,0.600000,6,0.400000,0.666667


In [38]:
#saving product_features to disk
product_features.to_csv('product_features.csv',index=False)

# Add to cart order features

In [130]:
#extracting atc_reordered,the count of number of reorders for each add_to_cart_order
atc_reordered=user_order_products_all_details.groupby(['add_to_cart_order']).agg({'reordered':sum}).reset_index()
atc_reordered.columns=['add_to_cart_order','atc_reordered']
atc_reordered

,add_to_cart_order,atc_reordered
0,1,2178183
1,2,2068060
2,3,1889311
3,4,1696923
4,5,1507665
...,...,...
140,141,0
141,142,0
142,143,0
143,144,0


In [132]:
#extracting atc_reordered_prop, the proportion of reorders out of all purchases for an add_to_cart_order
atc_reordered_prop=pd.DataFrame(atc_reordered)
atc_reordered_cnt=user_order_products_all_details.groupby(['add_to_cart_order']).agg({'reordered':'count'}).reset_index().reordered
atc_reordered_prop['atc_reordered_cnt']=atc_reordered_cnt
atc_reordered_prop['atc_reordered_prop']=atc_reordered_prop['atc_reordered']/atc_reordered_prop['atc_reordered_cnt']
atc_reordered_prop.drop(['atc_reordered','atc_reordered_cnt'],axis=1,inplace=True)
atc_reordered_prop

,add_to_cart_order,atc_reordered_prop
0,1,0.677533
1,2,0.676251
2,3,0.658037
3,4,0.636958
4,5,0.617383
...,...,...
140,141,0.000000
141,142,0.000000
142,143,0.000000
143,144,0.000000


In [133]:
#combining all atc_features
atc_features=pd.DataFrame({'add_to_cart_order':atc_reordered.add_to_cart_order,
                          'atc_reordered':atc_reordered.atc_reordered,
                           'atc_reordered_prop':atc_reordered_prop.atc_reordered_prop
                          })
atc_features

,add_to_cart_order,atc_reordered,atc_reordered_prop
0,1,2178183,0.677533
1,2,2068060,0.676251
2,3,1889311,0.658037
3,4,1696923,0.636958
4,5,1507665,0.617383
...,...,...,...
140,141,0,0.000000
141,142,0,0.000000
142,143,0,0.000000
143,144,0,0.000000


In [135]:
#saving atc_features to disk
atc_features.to_csv('atc_features.csv',index=False)

# Aisle features

In [137]:
#extracting aisle_reordered, the count of number of reorders for each aisle
aisle_reordered=user_order_products_all_details.groupby('aisle_id').agg({'reordered':sum}).reset_index()
aisle_reordered.columns=['aisle_id','aisle_reordered']
aisle_reordered

,aisle_id,aisle_reordered
0,1,42912
1,2,40365
2,3,272922
3,4,98243
4,5,17542
...,...,...
129,130,79003
130,131,120803
131,132,1308
132,133,6410


In [139]:
#extracting aisle_reordered_prop, the proportion  of reorders for each aisle
aisle_reordered_prop=pd.DataFrame(aisle_reordered)
aisle_reord_cnt=user_order_products_all_details.groupby('aisle_id').agg({'reordered':'count'}).reset_index().reordered
aisle_reordered_prop['aisle_reord_cnt']=aisle_reord_cnt
aisle_reordered_prop['aisle_reordered_prop']=aisle_reordered_prop['aisle_reordered']/aisle_reordered_prop['aisle_reord_cnt']
aisle_reordered_prop.drop(['aisle_reordered','aisle_reord_cnt'],axis=1,inplace=True)
aisle_reordered_prop

,aisle_id,aisle_reordered_prop
0,1,0.596597
1,2,0.489326
2,3,0.598007
3,4,0.489533
4,5,0.280627
...,...,...
129,130,0.499501
130,131,0.453062
131,132,0.212062
132,133,0.343093


In [140]:
#combining all aisle_features
aisle_features=pd.DataFrame({'aisle_id':aisle_reordered_prop.aisle_id,
                            'aisle_reordered':aisle_reordered.aisle_reordered,
                            'aisle_reordered_prop':aisle_reordered_prop.aisle_reordered_prop})
aisle_features

,aisle_id,aisle_reordered,aisle_reordered_prop
0,1,42912,0.596597
1,2,40365,0.489326
2,3,272922,0.598007
3,4,98243,0.489533
4,5,17542,0.280627
...,...,...,...
129,130,79003,0.499501
130,131,120803,0.453062
131,132,1308,0.212062
132,133,6410,0.343093


In [141]:
#saving aisle_features to disk
aisle_features.to_csv('aisle_features.csv',index=False)

# Department features

In [142]:
#extracting dep_reordered, the count of number of reorders for each department
dep_reordered=user_order_products_all_details.groupby('department_id').agg({'reordered':sum}).reset_index()
dep_reordered.columns=['department_id','dep_reordered']
dep_reordered

,department_id,dep_reordered
0,1,1211890
1,2,14806
2,3,739188
3,4,6160710
4,5,87595
5,6,99416
6,7,1757892
7,8,58760
8,9,399581
9,10,19950


In [144]:
#extracting dep_reordered_prop, the proportion  of reorders for each department
dep_reordered_prop=pd.DataFrame(dep_reordered)
dep_reord_cnt=user_order_products_all_details.groupby('department_id').agg({'reordered':'count'}).reset_index().reordered
dep_reordered_prop['dep_reord_cnt']=dep_reord_cnt
dep_reordered_prop['dep_reordered_prop']=dep_reordered_prop['dep_reordered']/dep_reordered_prop['dep_reord_cnt']
dep_reordered_prop.drop(['dep_reordered','dep_reord_cnt'],axis=1,inplace=True)
dep_reordered_prop

,department_id,dep_reordered_prop
0,1,0.541885
1,2,0.407980
2,3,0.628141
3,4,0.649913
4,5,0.569924
5,6,0.369229
6,7,0.653460
7,8,0.601285
8,9,0.461076
9,10,0.577040


In [147]:
#combining all department_features
dep_features=pd.DataFrame({'department_id':dep_reordered_prop.department_id,
                            'dep_reordered':dep_reordered.dep_reordered,
                            'dep_reordered_prop':dep_reordered_prop.dep_reordered_prop})
dep_features

,department_id,dep_reordered,dep_reordered_prop
0,1,1211890,0.541885
1,2,14806,0.407980
2,3,739188,0.628141
3,4,6160710,0.649913
4,5,87595,0.569924
5,6,99416,0.369229
6,7,1757892,0.653460
7,8,58760,0.601285
8,9,399581,0.461076
9,10,19950,0.577040


In [148]:
#saving department features to disk
dep_features.to_csv('dep_features.csv',index=False)

# Order_dow features

In [11]:
#extracting order_dow_reordered, the number of reorderes that have happend on a particular day of the week
order_dow_reordered=user_order_products_all_details.groupby('order_dow').agg({'reordered':sum}).reset_index()
order_dow_reordered.columns=['order_dow','order_dow_reordered']
order_dow_reordered

,order_dow,order_dow_reordered
0,0,3634366
1,1,3421285
2,2,2487537
3,3,2253698
4,4,2238166
5,5,2506649
6,6,2584835


In [13]:
#extracting order_dow_reordered_prop, the number of reorderes that have happend on a particular day of the week
order_dow_reordered_prop=pd.DataFrame(order_dow_reordered)
order_dow_reordered_prop['total_purchases']=user_order_products_all_details.groupby('order_dow').agg({'reordered':'count'}).reset_index().reordered
order_dow_reordered_prop['order_dow_reordered_prop']=order_dow_reordered_prop['order_dow_reordered']/order_dow_reordered_prop['total_purchases']
order_dow_reordered_prop.drop(['order_dow_reordered','total_purchases'],axis=1,inplace=True)
order_dow_reordered_prop

,order_dow,order_dow_reordered_prop
0,0,0.585276
1,1,0.603843
2,2,0.589771
3,3,0.586272
4,4,0.590979
5,5,0.595470
6,6,0.574369


In [14]:
#combining all order_dow features
order_dow_features=pd.DataFrame({'order_dow':order_dow_reordered_prop.order_dow,
                                 'order_dow_reordered':order_dow_reordered.order_dow_reordered,
                                 'order_dow_reordered_prop':order_dow_reordered_prop.order_dow_reordered_prop
                                })
order_dow_features

,order_dow,order_dow_reordered,order_dow_reordered_prop
0,0,3634366,0.585276
1,1,3421285,0.603843
2,2,2487537,0.589771
3,3,2253698,0.586272
4,4,2238166,0.590979
5,5,2506649,0.595470
6,6,2584835,0.574369


In [15]:
#saving order_dow_features to disk
order_dow_features.to_csv('order_dow_features.csv',index=False)

# Order_hour_of_day features

In [17]:
#extracting order_hod_reordered, the number of reorderes that have happend on a particular hour of the day
order_hod_reordered=user_order_products_all_details.groupby('order_hour_of_day').agg({'reordered':sum}).reset_index()
order_hod_reordered.columns=['order_hour_of_day','order_hod_reordered']
order_hod_reordered.head(5)

,order_hour_of_day,order_hod_reordered
0,0,123815
1,1,64506
2,2,38498
3,3,28714
4,4,30435


In [18]:
#extracting order_hod_reordered_prop, the number of reorderes that have happend on a particular hour of the day
order_hod_reordered_prop=pd.DataFrame(order_hod_reordered)
order_hod_reordered_prop['total_purchases']=user_order_products_all_details.groupby('order_hour_of_day').agg({'reordered':'count'}).reset_index().reordered
order_hod_reordered_prop['order_hod_reordered_prop']=order_hod_reordered_prop['order_hod_reordered']/order_hod_reordered_prop['total_purchases']
order_hod_reordered_prop.drop(['order_hod_reordered','total_purchases'],axis=1,inplace=True)
order_hod_reordered_prop

,order_hour_of_day,order_hod_reordered_prop
0,0,0.565500
1,1,0.557114
2,2,0.554455
3,3,0.559498
4,4,0.571195
5,5,0.607947
6,6,0.636497
7,7,0.644473
8,8,0.631883
9,9,0.619449


In [20]:
#combining all order_hod features
order_hod_features=pd.DataFrame({'order_hour_of_day':order_hod_reordered_prop.order_hour_of_day,
                                 'order_hod_reordered':order_hod_reordered.order_hod_reordered,
                                 'order_hod_reordered_prop':order_hod_reordered_prop.order_hod_reordered_prop
                                })
order_hod_features.head(5)

,order_hour_of_day,order_hod_reordered,order_hod_reordered_prop
0,0,123815,0.565500
1,1,64506,0.557114
2,2,38498,0.554455
3,3,28714,0.559498
4,4,30435,0.571195


In [21]:
#saving order_hour_of_day features to disk
order_hod_features.to_csv('order_hod_features.csv',index=False)

# Other Features

In [25]:
#extracting order_dow_hod_reord_count, the number of reorders for a particular day of the week and particular hour of the day
order_dow_hod_reord_count=user_order_products_all_details.groupby(['order_dow','order_hour_of_day']).agg({'reordered':sum}).reset_index()
order_dow_hod_reord_count.columns=['order_dow','order_hour_of_day','order_dow_hod_reord_count']
order_dow_hod_reord_count.head(5)

,order_dow,order_hour_of_day,order_dow_hod_reord_count
0,0,0,22412
1,0,1,12428
2,0,2,7028
3,0,3,5040
4,0,4,4576


In [27]:
#extracting order_dow_hod_reord_prop, the proportion of reorders for a particular day of the week and particular hour of the day
order_dow_hod_reord_prop=pd.DataFrame(order_dow_hod_reord_count)
order_dow_hod_reord_prop['order_dow_hod_reord_cnt']=user_order_products_all_details.groupby(['order_dow','order_hour_of_day']).agg({'reordered':'count'}).reset_index().reordered
order_dow_hod_reord_prop['order_dow_hod_reord_prop']=order_dow_hod_reord_prop['order_dow_hod_reord_count']/order_dow_hod_reord_prop['order_dow_hod_reord_cnt']
order_dow_hod_reord_prop.drop(['order_dow_hod_reord_count','order_dow_hod_reord_cnt'],axis=1,inplace=True)
order_dow_hod_reord_prop

,order_dow,order_hour_of_day,order_dow_hod_reord_prop
0,0,0,0.576500
1,0,1,0.551792
2,0,2,0.552299
3,0,3,0.569427
4,0,4,0.546584
...,...,...,...
163,6,19,0.564224
164,6,20,0.575499
165,6,21,0.586897
166,6,22,0.584808


In [28]:
#combining order_dow_hod_features
order_dow_hod_features=pd.DataFrame({'order_dow':order_dow_hod_reord_prop.order_dow,
                                     'order_hour_of_day':order_dow_hod_reord_prop.order_hour_of_day,
                                     'order_dow_hod_reord_count':order_dow_hod_reord_count.order_dow_hod_reord_count,
                                     'order_dow_hod_reord_prop':order_dow_hod_reord_prop.order_dow_hod_reord_prop
                                    })
order_dow_hod_features.head()

,order_dow,order_hour_of_day,order_dow_hod_reord_count,order_dow_hod_reord_prop
0,0,0,22412,0.576500
1,0,1,12428,0.551792
2,0,2,7028,0.552299
3,0,3,5040,0.569427
4,0,4,4576,0.546584


In [29]:
#saving order_dow_hod_features to disk
order_dow_hod_features.to_csv('order_dow_hod_features.csv',index=False)

In [20]:
#calculating ono_dsp_reord, the number of reorders for a particular order_number and days_since_prior_order
ono_dsp_reord=user_order_products_all_details.groupby(['order_number','days_since_prior_order']).agg({'reordered':sum}).reset_index()
ono_dsp_reord.columns=['order_number','days_since_prior_order','ono_dsp_reord']
ono_dsp_reord.head(5)

,order_number,days_since_prior_order,ono_dsp_reord
0,2,0.0,11853
1,2,1.0,5860
2,2,2.0,7812
3,2,3.0,11821
4,2,4.0,16942


In [23]:
#calculating ono_dsp_reord_prop, the ratio of reorders out of total purchases occuring on a particular order_number and days_since_prior_order
ono_dsp_reord_prop=pd.DataFrame(ono_dsp_reord)
ono_dsp_reord_prop['total_purchases']=user_order_products_all_details.groupby(['order_number','days_since_prior_order']).agg({'reordered':'count'}).reset_index().reordered
ono_dsp_reord_prop['ono_dsp_reord_prop']=ono_dsp_reord_prop['ono_dsp_reord']/ono_dsp_reord_prop['total_purchases']
ono_dsp_reord_prop.drop(['ono_dsp_reord','total_purchases'],axis=1,inplace=True)
ono_dsp_reord_prop

,order_number,days_since_prior_order,ono_dsp_reord_prop
0,2,0.0,0.576396
1,2,1.0,0.184899
2,2,2.0,0.168029
3,2,3.0,0.191582
4,2,4.0,0.220610
...,...,...,...
2819,99,15.0,0.882353
2820,99,16.0,1.000000
2821,99,17.0,0.800000
2822,99,18.0,0.961538


In [24]:
#combining all order_number, days_since_prior order features
ono_dsp_features=pd.DataFrame({'order_number':ono_dsp_reord_prop.order_number,
                                'days_since_prior_order':ono_dsp_reord_prop.days_since_prior_order,
                                'ono_dsp_reord':ono_dsp_reord.ono_dsp_reord,
                                'ono_dsp_reord_prop':ono_dsp_reord_prop.ono_dsp_reord_prop
                              })
ono_dsp_features

,order_number,days_since_prior_order,ono_dsp_reord,ono_dsp_reord_prop
0,2,0.0,11853,0.576396
1,2,1.0,5860,0.184899
2,2,2.0,7812,0.168029
3,2,3.0,11821,0.191582
4,2,4.0,16942,0.220610
...,...,...,...,...
2819,99,15.0,15,0.882353
2820,99,16.0,10,1.000000
2821,99,17.0,4,0.800000
2822,99,18.0,25,0.961538


In [25]:
#saving order_number, days_since_prior_order_features to disk
ono_dsp_features.to_csv('ono_dsp_features.csv',index=False)

# Summary

We came up with several user features, product features, user and product combined features and so on. We came up with different reordered counts for several different column combinations